# التعامل مع النصوص #

سوف نستعرض هنا كيفية تحضير النصوص العربية لاستخدامها في تدريب نماذج اللغة الكبيرة.

سوف نقوم في البداية بتحويل النص إلى ترميزات، حيث تمثل كل ترميزة كلمة أو جزء من كلمة. بعد ذلك نقوم بتشفير الترميزات باستخدام متجهات متعددة الأبعاد تسمى تضمينات، يمكن استخدامها لتدريب نماذج اللغة الكبيرة.

تجدر الأشارة إلى أن النص العربي يختلف عن غيره من اللغات مثل الإنجليزية بأنه يكتب من اليمين إلى اليسار و باحتوائه على الحركات أو التشكيل، مما يتطلب التعامل معه بشكل مختلف في بعض الأمور.

 ## تضمينات الكلمات
يتعذر على نماذج التعليم العميق، بما في ذلك نماذج اللغة الكبيرة، من معالجة النصوص بشكل مباشر. لذلك يجب علينا تمثيل الكلمات كمتجهات متعددة الأبعاد تسمى تضمينات، ويُشار إلى هذه العملية باسم تضمين الكلمات.


 ## ترميز النص
للبدء بتضمين الكلمات، يجب علينا أولا تقسيم النص إلى أجزاء تسمى ترميزات.

لنرى كيف يمكن تجزئة جملة قصيرة باستخدام الدالية  `re.split` مع الحفاظ على الفواصل:

In [1]:
import re

RE_ENCODER = r'([،.؟!:؛«»—]|\s)'
RE_DECODER = r'\s+([،.؟!:؛—])'

text = "مرحبا، بالعالم. هل هذا— اختبار؟"
result = re.split(RE_ENCODER, text)
result = [item.strip() for item in result if item.strip()]
print(result)
print(result[0])
print(result[-1])

['مرحبا', '،', 'بالعالم', '.', 'هل', 'هذا', '—', 'اختبار', '؟']
مرحبا
؟


لاحظ كيف يظهر النص العربي من اليمين إلى اليسار، بينما يتم تخزين السلسلة بالترتيب الذي تمت كتابتها به.

قي Python، `str` عبارة عن تسلسل من نقاط رموز Unicode مخزنة بالترتيب الذي تمت كتابتها به (الترتيب المنطقي)، لذا فإن `result[0] = مرحبا` و`result[-1] = ؟` بغض النظر عما إذا كان يتم عرضها من اليمين إلى اليسار أو من اليسار إلى اليمين.



النص الذي سنقوم بتحويله إلى رموز لتدريب LLM هو ”مغامرة العميل المرموق“، وهو متوفر في المجال العام وبالتالي يُسمح باستخدامه في مهام تدريب LLM.

النص متاح على Researchdata.se كجزء من [مجموعة الكتب الإلكترونية العربية](https://doi.org/10.5878/7rbh-gy93).


In [2]:
import os
import requests

file_path = "مغامرة-العميل-المرموق.txt"

if not os.path.exists(file_path):
    url = (
        "https://raw.githubusercontent.com/Abbazone/"
        "llm-from-scratch/main/ch02/01_main-chapter-code/"
        "العميل.txt"
    )
    response = requests.get(url)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)

In [3]:
with open(file_path, "r", encoding='utf-8') as f:
    raw_text = f.read()
print(f'Total number of characters: {len(raw_text)}')
print(raw_text[:500])

Total number of characters: 44740
مغامرة العميل المرموق

تأليف
آرثر كونان دويل

ترجمة
دينا عادل غراب

مراجعة
شيماء طه الريدي

مغامرة العميل المرموق

حين طلبتُ الإذنَ من السيد هولمز، للمرة العاشرة خلال عدة سنوات، للبوح بالقصة التالية، أجابني بقوله: «لا ضررَ من ذلك الآن.» لأحصل بذلك أخيرًا على الإذن بتدوينِ ما كان — من بضع نواحٍ — اللحظةَ الأبرز والأهم في مسيرة صديقي المهنية ذات يوم.

كان لدينا، أنا وهولمز ضَعْفٌ تجاهَ الحَمَّام التركي؛ فلم أجده أقلَّ تحفظًا وأكثر آدميةً كما كان وسطَ البخار في أجواء التراخي الممتعة في حجرة التجفيف


لنختبر المرمز الذي كتبناه على هذا النص الكبير نسبيا:

In [4]:
preprocessed = re.split(RE_ENCODER, raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(f'Number of tokens in the text: {len(preprocessed)}')
print(f'Number of unique tokens in the text: {len(set(preprocessed))}')
print(f'First 30 tokens in the text:\n{preprocessed[:100]}')

Number of tokens in the text: 9399
Number of unique tokens in the text: 3890
First 30 tokens in the text:
['مغامرة', 'العميل', 'المرموق', 'تأليف', 'آرثر', 'كونان', 'دويل', 'ترجمة', 'دينا', 'عادل', 'غراب', 'مراجعة', 'شيماء', 'طه', 'الريدي', 'مغامرة', 'العميل', 'المرموق', 'حين', 'طلبتُ', 'الإذنَ', 'من', 'السيد', 'هولمز', '،', 'للمرة', 'العاشرة', 'خلال', 'عدة', 'سنوات', '،', 'للبوح', 'بالقصة', 'التالية', '،', 'أجابني', 'بقوله', ':', '«', 'لا', 'ضررَ', 'من', 'ذلك', 'الآن', '.', '»', 'لأحصل', 'بذلك', 'أخيرًا', 'على', 'الإذن', 'بتدوينِ', 'ما', 'كان', '—', 'من', 'بضع', 'نواحٍ', '—', 'اللحظةَ', 'الأبرز', 'والأهم', 'في', 'مسيرة', 'صديقي', 'المهنية', 'ذات', 'يوم', '.', 'كان', 'لدينا', '،', 'أنا', 'وهولمز', 'ضَعْفٌ', 'تجاهَ', 'الحَمَّام', 'التركي', '؛', 'فلم', 'أجده', 'أقلَّ', 'تحفظًا', 'وأكثر', 'آدميةً', 'كما', 'كان', 'وسطَ', 'البخار', 'في', 'أجواء', 'التراخي', 'الممتعة', 'في', 'حجرة', 'التجفيف', '.', 'يوجد', 'في', 'الدور']



## تحويل الترميزات إلى معرفات

في هذه الخطوة سوف نقوم بتحويل الترميزات إلى أعداد تسمي معرفات الترميزات. يتم ذلك من خلال إنشاء مفردات تحدد كيفية تعيين كل ترميزه إلى عدد صحيح فريد.


In [5]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(f'Vocabulary size: {vocab_size}')

Vocabulary size: 3890


In [6]:
str_to_int = {token: i for i, token in enumerate(all_words)}    #
for i, item in enumerate(str_to_int.items()):
    print(f'{i}: {item}')
    if i >= 50:
        break

0: ('!', 0)
1: ('.', 1)
2: (':', 2)
3: ('«', 3)
4: ('»', 4)
5: ('،', 5)
6: ('؛', 6)
7: ('؟', 7)
8: ('آبَهُ', 8)
9: ('آتٍ', 9)
10: ('آثار', 10)
11: ('آجلًا', 11)
12: ('آخر', 12)
13: ('آخرون', 13)
14: ('آخرين', 14)
15: ('آخِر', 15)
16: ('آدميةً', 16)
17: ('آرثر', 17)
18: ('آلت', 18)
19: ('آن', 19)
20: ('آنسة', 20)
21: ('آه', 21)
22: ('أبديت', 22)
23: ('أبراج', 23)
24: ('أبرهن', 24)
25: ('أبوها', 25)
26: ('أبي', 26)
27: ('أبيها', 27)
28: ('أتابع', 28)
29: ('أتابِعَ', 29)
30: ('أتاح', 30)
31: ('أتحدث', 31)
32: ('أتحرى', 32)
33: ('أتخيل', 33)
34: ('أتسألني', 34)
35: ('أتعابَك', 35)
36: ('أتقصد', 36)
37: ('أتوقَّع', 37)
38: ('أتيت', 38)
39: ('أتينا', 39)
40: ('أثارت', 40)
41: ('أثر', 41)
42: ('أثره', 42)
43: ('أثرٍ', 43)
44: ('أثرِ', 44)
45: ('أجابني', 45)
46: ('أجبته', 46)
47: ('أجد', 47)
48: ('أجده', 48)
49: ('أجدها', 49)
50: ('أجرة', 50)


بالمقابل نحتاج أيضا إلى طريقة لتحويل معرفات الترميزات إلى الترميزات الاصلية:

In [7]:
int_to_str = {i: s for s, i in str_to_int.items()}
print(int_to_str[50])

أجرة



سنقوم الآن باستخدام فئة بايثون لكتابة مرمز متكامل بالمميزات التالية:

- دالية التشفير: تقوم بتقسيم النص إلى ترميزات ومن ثم تحويلها إلى معرفات.

 - دالية فك التشفير: تعمل بالعكس، أي تقوم بتحويل المعرفات إلى ترميزات مرة أخرى.



In [8]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(RE_ENCODER, text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = [self.int_to_str[i] for i in ids]
        text = ' '.join(text)
        text = re.sub(RE_DECODER, r'\1', text)    # remove extra spaces before punctuation.
        text = re.sub(r"«\s+", "«", text)
        text = re.sub(r"\s+»", "»", text)
        return text


لنختبر دالية التشفير في المرمز لتحويل النص إلى أعداد:

In [9]:
tokenizer = SimpleTokenizerV1(vocab=str_to_int)
text = """حين طلبتُ الإذنَ من السيد هولمز، للمرة العاشرة خلال عدة سنوات، للبوح بالقصة التالية، أجابني بقوله: «لا ضررَ من ذلك الآن.»"""
ids = tokenizer.encode(text)
print(ids)

[1898, 2382, 524, 3191, 787, 3351, 5, 2919, 869, 1943, 2423, 2187, 5, 2900, 1318, 579, 5, 45, 1446, 2, 3, 2803, 2349, 3191, 2014, 458, 1, 4]



ثم لنختبر دالية فك التشفير لتحويل الأعداد إلى ترميزات:

In [10]:
print(tokenizer.decode(ids))

حين طلبتُ الإذنَ من السيد هولمز، للمرة العاشرة خلال عدة سنوات، للبوح بالقصة التالية، أجابني بقوله: «لا ضررَ من ذلك الآن.»



بناء على هذا نستنتج أن عملية التشفير وفك التشفير تمت بنجاح.

لكن ماذا يحدث إذا صادف المرمز كلمات جديدة لم تكن موجودة عند بناء المفردات؟ لنتابع المثال التالي؟

In [11]:
text = "مرحبا، يوم سعيد؟"
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

KeyError: 'مرحبا'

سبب هذا الخطأ هو أن كلمة "مرحبا" كلمة جديدة، أي ليست موجودة في النص الأصلي، وبالتالي ليست موجودة في المفردات. سنرى كيف يمكن التغلب على هذه المشكلة.

## إضافة الترميزات الخاصة
سنقوم بإضافة بعض التعديلات على المرمز ليتمكن من التعامل مع الكلمات الجديدة. سنضيف أيضا تعديلا يساعد النموذج اللغوي على فهم سياق النص من خلال الفصل بين النصوص المتتابعة.
سنقوم بتعديل المفردات والمرمز لدعم ترميزين جديدين هما `<|unk|>` و`<|endoftext|>`.

In [13]:
all_tokens = sorted(set(preprocessed))
print(f'Original vocabulary size: {len(str_to_int)}')
all_tokens.extend(['<|unk|>', '<|endoftext|>'])
str_to_int = {s: i for i, s in enumerate(all_tokens)}
print(f'Extended vocabulary size: {len(str_to_int)}')

Original vocabulary size: 3890
Extended vocabulary size: 3892


In [14]:
for s, i in list(str_to_int.items())[-5:]:
    print(f'{s}: {i}')

—: 3887
…: 3888
ﻟ: 3889
<|unk|>: 3890
<|endoftext|>: 3891


تم دمج الترميزتين الجديدتين في المفردات بنجاح.


سنقوم الآن بإضافة التعديلات التالية على فئة بايثون السابقة:
- دالية التشفير: تقوم بتقسيم النص إلى ترميزات ومن ثم تحويلها إلى معرفات.
- يتم إرسال الكلمات الجديدة إلى ترميزة خاصة `<|unk|>`
- يتم الفصل بين النصوص المختلفة باستخدام ترميزة خاصة `<|endoftext|>`.
 - دالية فك التشفير: تعمل بالعكس، أي تقوم بتحويل المعرفات إلى ترميزات مرة أخرى.


In [15]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(RE_ENCODER, text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item.strip() if item in self.str_to_int else '<|unk|>' for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = [self.int_to_str[i] for i in ids]
        text = ' '.join(text)
        text = re.sub(RE_DECODER, r'\1', text)    # remove extra spaces before punctuation.
        text = re.sub(r"«\s+", "«", text)
        text = re.sub(r"\s+»", "»", text)
        return text


لنختبر المرمز المعدل على الجملة السابقة:


In [16]:
text = "مرحبا، يوم سعيد؟"
tokenizer = SimpleTokenizerV2(vocab=str_to_int)
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

[3890, 5, 3852, 2172, 7]
<|unk|>، يوم سعيد؟


نلاحظ أنه تم تعيين الترميزة `<|unk|>` لكلمة "مرحبا" بنجاح.

لنختبر الآن المرمز على جملتين مختلفتين:

In [1]:
text1 = "مرحبا، يوم سعيد؟"
text2 = "حين طلبتُ، كلمةجديدة، من السيد هولمز"

text = " <|endoftext|> ".join((text1, text2))

print(text)

مرحبا، يوم سعيد؟ <|endoftext|> حين طلبتُ، كلمةجديدة، من السيد هولمز


In [52]:
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

[3890, 5, 3852, 2172, 7, 3891, 1898, 2382, 5, 3890, 5, 3191, 787, 3351]
<|unk|>، يوم سعيد؟ <|endoftext|> حين طلبتُ، <|unk|>، من السيد هولمز


نرى أن المرمز تعرف على الكلمات الجديدة مثل "مرحبا" و "كلمةجديدة"، وأيضا الترميزة الخاصة `<|endoftext|>` بنجاح.

## 2.5 Byte pair encoding

The Byte Pair Encoder (BPE) was used to train LLMs such as GPT-2, GPT-3, and the original model used in ChatGPT.

Let's first look at an existing implementation from the tiktoken library:

In [17]:
import tiktoken

tokenizer = tiktoken.get_encoding('gpt2')

text1 = "مرحبا، يوم سعيد؟"
text2 = "حين طلبتُ، كلمةجديدة، من السيد هولمز"
text = " <|endoftext|> ".join((text1, text2))

print(text)

مرحبا، يوم سعيد؟ <|endoftext|> حين طلبتُ، كلمةجديدة، من السيد هولمز


In [27]:
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)
strings = tokenizer.decode(integers)
print(strings)

[25405, 26897, 148, 255, 39848, 12919, 148, 234, 18923, 232, 30335, 25405, 17550, 111, 44690, 22654, 38843, 148, 253, 220, 50256, 17550, 255, 22654, 23338, 17550, 115, 13862, 39848, 41486, 149, 237, 148, 234, 18923, 225, 13862, 25405, 45632, 148, 105, 38843, 22654, 38843, 45632, 148, 234, 47048, 23338, 28981, 45692, 22654, 38843, 18923, 229, 30335, 13862, 25405, 148, 110]
مرحبا، يوم سعيد؟ <|endoftext|> حين طلبتُ، كلمةجديدة، من السيد هولمز


The encoding and decoding looks good.

Specifically, we see that the BPE tokenizer managed to encode and decode unknown words such as كلمةجديدة correctly.

This is because the algorithm underlying BPE breaks down words that aren't in its predefined vocabulary into smaller subword units or even individual characters, enabling it to handle out-of-vocabulary words.

In [28]:
print(tokenizer.encode('مرحبا'))
print(tokenizer.encode('كلمةجديدة'))
print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))

[25405, 26897, 148, 255, 39848, 12919]
[149, 225, 13862, 25405, 45632, 148, 105, 38843, 22654, 38843, 45632]
[50256]


But notice also the large number of token IDs compared to the previous approach based on `re.split`.

To understand why, let's take a closer look at how the GPT-2 tokenization works:
1. Convert the text to UTF-8 bytes
2. Map bytes through a reversible “byte encoder”
3. Iteratively apply BPE merges to combine frequent byte sequences into tokens

For English text, many common sequences have merges, so you get big tokens like " hello" or "ing".

For Arabic (and many non-Latin scripts), GPT-2’s merges are much weaker because the GPT-2 vocab was built from data that was heavily skewed toward English/Latin text. So Arabic often falls back to smaller byte chunks, meaning more tokens. Furthermore, the presence of diacritics such as Tashkil (تشكيل) or Harakat (حركات) for vowels, like for example in طلبتُ which includes damma/tanween, is treated as a separate Unicode point. This often breaks BPE merges which leads to more tokens.